# Exploratory Data Analysis

Importação, unificação e limpeza das Bases de Dados

In [ ]:
from google.colab import files

# Fazer upload manual do arquivo .xlsx
uploaded = files.upload()


Saving atlasivs_dadosbrutos_pt_v2.xlsx to atlasivs_dadosbrutos_pt_v2.xlsx


In [ ]:
import pandas as pd

# Carregar o arquivo que foi feito upload
ivs = pd.read_excel("atlasivs_dadosbrutos_pt_v2.xlsx")

In [ ]:
# Informações gerais do DataFrame
ivs.info()

# Ver quantos valores nulos existem por coluna
ivs.isnull().sum().sort_values(ascending=False)

# Ver um resumo estatístico das variáveis numéricas
ivs.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340786 entries, 0 to 340785
Columns: 103 entries, id to t_nremunerado_18m
dtypes: float64(2), int64(2), object(99)
memory usage: 267.8+ MB


,id,ano,ivs,idhm
count,340786.000000,340786.000000,140312.000000,277444.000000
mean,190671.979342,2005.582078,0.370811,0.608250
std,127660.960189,5.531609,0.137952,0.123754
min,1.000000,2000.000000,0.034000,0.000000
25%,95079.250000,2000.000000,0.266000,0.533000
50%,180275.500000,2010.000000,0.357000,0.620000
75%,265471.750000,2010.000000,0.467000,0.697000
max,660006.000000,2022.000000,0.872000,0.965000


Estrutura da base:

A base possui 340.786 observações (linhas) e 103 variáveis (colunas).

Entre essas variáveis, 99 estão no formato object, o que indica que seus valores estão armazenados como texto (muito provavelmente com vírgulas no lugar de pontos decimais), o que impede análises numéricas até que sejam convertidas.

Presença de valores ausentes:

Identificamos que várias variáveis importantes possuem muitos valores nulos (NaN).

Por exemplo:

A variável ivs (Índice de Vulnerabilidade Social) aparece em apenas 140.312 das 340.786 linhas.

A variável idhm (Índice de Desenvolvimento Humano Municipal) está presente em 277.444 linhas, ainda assim com uma quantidade considerável de ausências.

Resumo estatístico preliminar:

A função describe() permitiu observar que os valores de ivs variam entre 0.034 e 0.872, e o idhm varia entre 0.000 e 0.965.

As médias de ivs e idhm foram de aproximadamente 0.37 e 0.61, respectivamente, o que indica padrões médios de vulnerabilidade e desenvolvimento moderados, com uma ampla variação entre os territórios analisados.

Será necessário:

Selecionar apenas as colunas relevantes para o TCC;

Converter os dados de texto para números reais (float);

Tratar os valores ausentes antes de qualquer unificação com outras bases.

In [ ]:
# Selecionando variáveis importantes para o TCC
colunas_usar = [
    'ano', 'municipio',
    'ivs', 'ivs_infraestrutura_urbana', 'ivs_capital_humano', 'ivs_renda_e_trabalho',
    'idhm', 'idhm_educ', 'idhm_renda', 'idhm_long',
    't_vulner', 't_desocup18m', 't_formal_18m', 't_analf_15m', 't_renda_trab'
]

ivs_filt = ivs[colunas_usar].copy()
ivs_filt.head()



,ano,municipio,ivs,ivs_infraestrutura_urbana,ivs_capital_humano,ivs_renda_e_trabalho,idhm,idhm_educ,idhm_renda,idhm_long,t_vulner,t_desocup18m,t_formal_18m,t_analf_15m,t_renda_trab
0,2000,geral,0.335,"0,285","0,400","0,321",0.703,"0,569","0,740","0,827","37,11","12,16","57,36","8,32","77,51"
1,2010,geral,0.255,"0,251","0,267","0,248",0.784,"0,708","0,791","0,860","21,40","5,89","65,13","5,76","73,56"
2,2011,geral,0.214,"0,197","0,228","0,215",0.790,"0,746","0,762","0,867","19,50","5,26","63,50","5,26","78,16"
3,2012,geral,0.200,"0,199","0,216","0,185",0.800,"0,754","0,777","0,875","15,61","4,75","64,65","5,27","77,34"
4,2013,geral,0.197,"0,200","0,210","0,182",0.806,"0,764","0,782","0,878","15,34","4,91","65,85","5,18","77,68"


Após a análise exploratória inicial, o próximo passo foi filtrar apenas as colunas mais importantes para os objetivos do TCC. Essa etapa tem como objetivo:

1- Reduzir o tamanho da base,

2- Focar nas variáveis que ajudam a explicar a demanda por serviços de atenção primária,

3- E facilitar os próximos passos de limpeza, unificação e análise.

As variáveis selecionadas foram:

| Variável                                       | Significado básico                                                                  |
| ---------------------------------------------- | ----------------------------------------------------------------------------------- |
| `ano`                                          | Ano de referência dos dados                                                         |
| `municipio`                                    | Código do município (ou geral, no caso de agregações)                               |
| `ivs`                                          | Índice de Vulnerabilidade Social geral                                              |
| `ivs_infraestrutura_urbana`                    | Subíndice de infraestrutura (água, esgoto, coleta de lixo etc.)                     |
| `ivs_capital_humano`                           | Subíndice ligado a educação e saúde                                                 |
| `ivs_renda_e_trabalho`                         | Subíndice de renda e inserção no mercado de trabalho                                |
| `idhm`, `idhm_educ`, `idhm_renda`, `idhm_long` | Dimensões do Índice de Desenvolvimento Humano (geral, educação, renda, longevidade) |
| `t_vulner`                                     | Proporção da população com renda per capita ≤ ½ salário mínimo                      |
| `t_desocup18m`                                 | Taxa de desocupação entre pessoas com 18 anos ou mais                               |
| `t_formal_18m`                                 | Percentual de trabalhadores formais com 18 anos ou mais                             |
| `t_analf_15m`                                  | Taxa de analfabetismo entre maiores de 15 anos                                      |
| `t_renda_trab`                                 | Percentual da renda domiciliar proveniente do trabalho                              |


Próximo passo: converter todas as colunas para o tipo float

In [ ]:
# Corrigir colunas com vírgulas apenas se forem texto (object)
colunas_numericas = ivs_filt.columns.difference(['ano', 'municipio', 'nome_regiao'])

for col in colunas_numericas:
    if ivs_filt[col].dtype == 'object':
        # Replace commas with periods
        ivs_filt[col] = ivs_filt[col].str.replace(',', '.', regex=False)
        # Convert to numeric, coercing errors to NaN
        ivs_filt[col] = pd.to_numeric(ivs_filt[col], errors='coerce')

In [ ]:
# Verifica os tipos de dados para confirmar que agora são float
ivs_filt.dtypes


,0
ano,int64
municipio,object
ivs,float64
ivs_infraestrutura_urbana,float64
ivs_capital_humano,float64
ivs_renda_e_trabalho,float64
idhm,float64
idhm_educ,float64
idhm_renda,float64
idhm_long,float64


In [ ]:
# Ver quantos valores faltantes ainda existem por coluna
ivs_filt.isnull().sum().sort_values(ascending=False)


,0
ivs,200474
ivs_capital_humano,162454
ivs_infraestrutura_urbana,130687
idhm,63342
idhm_long,63342
idhm_educ,63339
ivs_renda_e_trabalho,58653
t_formal_18m,55410
t_renda_trab,55299
idhm_renda,55126


Após a seleção das variáveis relevantes, realizamos a conversão dos dados do formato textual (com vírgulas) para valores numéricos (float), utilizando o método str.replace(',', '.') seguido de pd.to_numeric().

Essa conversão foi necessária porque as colunas estavam originalmente no tipo object, o que inviabilizava qualquer tipo de análise estatística ou modelagem. Agora, todas as variáveis quantitativas estão corretamente no formato float64.

Com os dados convertidos, rodamos um diagnóstico com isnull().sum() para verificar a presença de valores ausentes (NaN).

Os resultados mostraram que:

Muitas colunas possuem dezenas de milhares de valores ausentes.

Por exemplo:

ivs: ~200.000 ausentes (de 340.786 linhas)

ivs_capital_humano: ~162.000 ausentes

idhm: ~63.000 ausentes

Apenas ano e municipio não têm valores nulos.

Proximos passos:

Filtrar para anos em que há muitos dados válidos, manter os anos com maior cobertura e qualidade de dados, podemos verificar quais anos possuem menos nulos e usar apenas esses

In [ ]:
# Verificar quantos valores válidos (não nulos) existem por variável em cada ano
ivs_filt.groupby("ano").count().sort_index(ascending=True)


,municipio,ivs,ivs_infraestrutura_urbana,ivs_capital_humano,ivs_renda_e_trabalho,idhm,idhm_educ,idhm_renda,idhm_long,t_vulner,t_desocup18m,t_formal_18m,t_analf_15m,t_renda_trab
ano,,,,,,,,,,,,,,
2000,161480,63172,96302,80934,130295,126733,126736,131650,126733,131838,131648,131411,132262,131503
2010,163299,71045,104652,89364,139528,136926,136926,140178,136926,140358,140327,140133,140361,140152
2011,1161,514,772,668,1055,1044,1044,1055,1044,1055,1055,1055,1055,1055
2012,1161,516,774,666,1051,1048,1048,1053,1048,1053,1053,1053,1053,1053
2013,1161,516,774,674,1060,1051,1051,1061,1051,1061,1061,1061,1061,1061
2014,1161,516,774,681,1066,1060,1060,1069,1060,1069,1069,1069,1069,1069
2015,1161,516,774,682,1066,1062,1062,1071,1062,1071,1071,1071,1071,1071
2016,1458,502,754,665,1001,1217,1217,1217,1217,1217,1217,1217,1217,1217
2017,1458,502,752,663,994,1202,1202,1202,1202,1202,1202,1202,1202,1202


Análise dos resultados
🟢 2000 e 2010

São os únicos anos com cobertura ampla para quase todas as variáveis:

ivs: ~63k (2000) / ~71k (2010)

idhm: ~130k (2000) / ~139k (2010)

t_vulner: ~131k (2000) / ~140k (2010)


🔴 2011–2022

Só ~1000 registros por ano.

Ruins para análise estatística confiável.


Com base na análise da cobertura dos dados ano a ano, optamos por manter apenas os anos 2000, 2010 e 2020. Essa decisão foi tomada por três razões principais:

1-Cobertura completa das principais variáveis (como IVS, IDHM e indicadores de vulnerabilidade e trabalho) nesses anos.

2-Possibilidade de observar a evolução dos determinantes sociais da saúde em um intervalo de 20 anos, com saltos decenais.

3-Alinhamento com ciclos censitários e de políticas públicas, facilitando análises longitudinais e comparativas.

In [ ]:
# Filtrar os anos desejados
anos_para_manter = [2000, 2010, 2020]
ivs_filtrado = ivs_filt[ivs_filt['ano'].isin(anos_para_manter)].copy()

# Verificar tamanho da nova base
ivs_filtrado.shape


(326237, 15)

In [ ]:
# Remover linhas com valores ausentes restantes
ivs_filtrado = ivs_filtrado.dropna()

# Confirmar o novo tamanho da base
ivs_filtrado.shape


(134556, 15)

Com isso, temos agora uma base filtrada, limpa e pronta para análise, contendo:

134.556 linhas

15 variáveis relevantes

Somente dos anos 2000, 2010 e 2020

E sem valores nulos (NaN)

Com isso, o DataFrame final passou a conter 134.556 registros completos, distribuídos entre os 3 anos selecionados, com 15 variáveis socioeconômicas e de desenvolvimento humano.


Proximo passo: Unificar com outras bases

Integrar com dados demográficos por bairro, distrito ou setor

Explorar como variáveis locais afetam os indicadores

In [ ]:
from google.colab import files

# Fazer upload manual do arquivo de bairros
uploaded = files.upload()


Saving Agregados_por_bairros_basico_BR.xlsx to Agregados_por_bairros_basico_BR.xlsx


In [ ]:
# Carregar a base de bairros enviada
bairros = pd.read_excel("Agregados_por_bairros_basico_BR.xlsx")

# Visualizar as primeiras linhas
bairros.head()


,CD_BAIRRO,NM_BAIRRO,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,...,CD_CONCURB,NM_CONCURB,AREA_KM2,v0001,v0002,v0003,v0004,v0005,v0006,v0007
0,1100015001,Centro,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,...,.,NaN,0.628243,1213,637,633,4,2.5,0.0146,480
1,1100015002,Liberdade,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,...,.,NaN,0.951916,1029,488,488,0,2.6,0.0281,391
2,1100015003,Cidade Alta,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,...,.,NaN,0.596322,1288,602,600,2,2.6,0.004,499
3,1100015005,Princesa Isabel,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,...,.,NaN,1.951955,2846,1242,1242,0,2.7,0.0179,1060
4,1100015006,Redondo,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,...,.,NaN,1.453434,2365,990,990,0,2.7,0.0619,873


Colunas relevantes para o merge:

| Coluna do DataFrame de bairros | Significado                                          |
| ------------------------------ | ---------------------------------------------------- |
| `CD_MUN`                       | Código do município (ex: 1100015) – **fundamental!** |
| `NM_MUN`                       | Nome do município (ex: Floresta D’Oeste)             |
| `NM_BAIRRO`                    | Nome do bairro – útil para análise local             |
| `v0001`, `v0002`, ...          | Variáveis demográficas agregadas por bairro          |


a base ivs_filtrado, a coluna municipio é equivalente ao CD_MUN da base de bairros.
Isso significa que podemos fazer o merge pelas colunas:

ivs_filtrado["municipio"]

bairros["CD_MUN"]


ATENÇÃO: Na base ivs_filtrado, a coluna municipio está no formato object e pode conter "geral" ou estar formatada diferente.

Proximo passo: Vamos converter tudo para string e preparar o merge com segurança.

In [ ]:
# Converter códigos para string para garantir correspondência
ivs_filtrado["municipio"] = ivs_filtrado["municipio"].astype(str)
bairros["CD_MUN"] = bairros["CD_MUN"].astype(str)

# Fazer o merge (junção) com base no código do município
ivs_com_bairros = pd.merge(ivs_filtrado, bairros, left_on="municipio", right_on="CD_MUN", how="inner")

# Verificar o resultado
ivs_com_bairros.head()


,ano,municipio,ivs,ivs_infraestrutura_urbana,ivs_capital_humano,ivs_renda_e_trabalho,idhm,idhm_educ,idhm_renda,idhm_long,...,CD_CONCURB,NM_CONCURB,AREA_KM2,v0001,v0002,v0003,v0004,v0005,v0006,v0007
0,2000,5217401,0.234,0.104,0.327,0.271,0.686,0.537,0.718,0.836,...,.,NaN,0.504719,1635,812,807,5,2.3,0.0074,680
1,2000,5217401,0.234,0.104,0.327,0.271,0.686,0.537,0.718,0.836,...,.,NaN,0.944571,2645,1175,1175,0,2.5,0.0028,1058
2,2000,5217401,0.234,0.104,0.327,0.271,0.686,0.537,0.718,0.836,...,.,NaN,0.225690,962,398,398,0,2.6,0,369
3,2000,5217401,0.234,0.104,0.327,0.271,0.686,0.537,0.718,0.836,...,.,NaN,0.276035,1198,483,480,3,2.7,0.0091,439
4,2000,5217401,0.234,0.104,0.327,0.271,0.686,0.537,0.718,0.836,...,.,NaN,0.944546,2946,1275,1271,4,2.6,0.0035,1138


Para enriquecer a base principal com informações geodemográficas, realizamos a junção da base ivs_filtrado com a base Agregados_por_bairros_basico_BR.xlsx, utilizando como chave comum o código do município (CD_MUN).

Esse merge permitiu incorporar informações complementares como:

Nome dos bairros e municípios,

Dados de área territorial (AREA_KM2),

Variáveis populacionais codificadas .

Esse cruzamento torna a base muito mais robusta para análises espaciais, demográficas e comparativas, especialmente no contexto da demanda por serviços de atenção primária à saúde.

Proximo passo: Carregar a base de distritos

In [ ]:
from google.colab import files

# Fazer upload manual da base de distritos
uploaded = files.upload()


Saving Agregados_por_distritos_demografia_BR.xlsx to Agregados_por_distritos_demografia_BR.xlsx


In [ ]:
# Carregar a base de distritos
distritos = pd.read_excel("Agregados_por_distritos_demografia_BR.xlsx")

# Visualizar as primeiras linhas
distritos.head()


,CD_DIST,NM_DIST,V01006,V01007,V01008,V01009,V01010,V01011,V01012,V01013,...,V01032,V01033,V01034,V01035,V01036,V01037,V01038,V01039,V01040,V01041
0,110001505,Alta Floresta D'Oeste,16699,8267,8432,592,598,589,645,612,...,1166,1144,1250,1191,1211,2485,2487,2028,1479,1053
1,110001515,Filadélfia d'Oeste,551,299,252,23,27,25,19,19,...,39,46,44,44,33,79,84,65,48,19
2,110001520,Izidolândia,532,287,245,25,18,17,25,12,...,44,42,42,34,44,85,78,62,37,22
3,110001525,Nova Gease d'Oeste,1071,562,509,35,32,43,57,27,...,90,79,104,73,92,126,120,130,82,54
4,110001530,Rolim de Moura do Guaporé,801,442,359,42,31,45,45,31,...,55,86,81,51,58,129,116,85,43,23


Estrutura da base de distritos

| Coluna            | Significado presumido                                                                   |
| ----------------- | --------------------------------------------------------------------------------------- |
| `CD_DIST`         | Código do distrito                                                                      |
| `NM_DIST`         | Nome do distrito                                                                        |
| `V01006`–`V01041` | Variáveis demográficas (como população, idade, etc.) – usar dicionário para interpretar |


Essa base não possui explicitamente o código do município (CD_MUN), mas o código do distrito (CD_DIST) começa com o código do município.

Exemplo:

CD_DIST = 110001505 → o código do município é 1100015



Estratégia para unir com a base ivs_filtrado

1-Vamos extrair os 7 primeiros dígitos de CD_DIST como CD_MUN

2-Converter ambos os códigos para string

3-Fazer o merge com ivs_filtrado usando municipio == CD_MUN

In [ ]:
# Extrair o código do município dos 7 primeiros dígitos de CD_DIST
distritos["CD_MUN"] = distritos["CD_DIST"].astype(str).str[:7]

# Converter tipo para garantir merge
ivs_filtrado["municipio"] = ivs_filtrado["municipio"].astype(str)
distritos["CD_MUN"] = distritos["CD_MUN"].astype(str)

# Fazer o merge
ivs_com_distritos = pd.merge(ivs_filtrado, distritos, left_on="municipio", right_on="CD_MUN", how="inner")

# Verificar o resultado
ivs_com_distritos.head()

,ano,municipio,ivs,ivs_infraestrutura_urbana,ivs_capital_humano,ivs_renda_e_trabalho,idhm,idhm_educ,idhm_renda,idhm_long,...,V01033,V01034,V01035,V01036,V01037,V01038,V01039,V01040,V01041,CD_MUN
0,2000,5300108,0.314,0.422,0.291,0.229,0.805,0.687,0.857,0.886,...,182575,207834,225156,220704,457174,460525,330213,205840,158123,5300108
1,2010,5300108,0.254,0.408,0.188,0.167,0.885,0.806,0.920,0.935,...,182575,207834,225156,220704,457174,460525,330213,205840,158123,5300108
2,2000,5200134,0.340,0.236,0.490,0.293,0.654,0.493,0.699,0.812,...,1448,1627,1562,1539,3195,3267,2758,1821,1331,5200134
3,2010,5200134,0.262,0.243,0.293,0.251,0.729,0.629,0.737,0.835,...,1448,1627,1562,1539,3195,3267,2758,1821,1331,5200134
4,2010,5200159,0.280,0.255,0.204,0.382,0.734,0.706,0.684,0.821,...,129,129,119,119,304,321,359,310,249,5200159


Para enriquecer a análise dos indicadores sociais com dados em nível de distrito, realizamos a união entre a base principal (ivs_filtrado) e a base Agregados_por_distritos_demografia_BR.xlsx, com base no código do município (CD_MUN), extraído dos sete primeiros dígitos do campo CD_DIST.

A união permitiu incorporar variáveis agregadas por distrito (ex: população por faixa etária, escolaridade, etc.), que serão úteis para analisar padrões de vulnerabilidade, desigualdade e demandas sociais na atenção primária à saúde.

Essa base unificada será uma das principais ferramentas do projeto para identificar correlações entre condições demográficas e indicadores de desenvolvimento, além de suportar análises de previsão de demanda por serviços públicos.


Proximo passo: Adicionar a ultima base de dados Agregados_por_setores_demografia_BR.xlsx

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Agregados_por_setores_demografia_BR.xlsx to Agregados_por_setores_demografia_BR.xlsx


In [ ]:
# Carregar a base de setores censitários
setores = pd.read_excel("Agregados_por_setores_demografia_BR.xlsx")

# Visualizar as primeiras linhas
setores.head()


,CD_setor,V01006,V01007,V01008,V01009,V01010,V01011,V01012,V01013,V01014,...,V01032,V01033,V01034,V01035,V01036,V01037,V01038,V01039,V01040,V01041
0,110001505000002,928,428,500,30,39,24,44,36,25,...,68,62,88,68,58,144,129,124,66,53
1,110001505000003,556,270,286,15,20,20,19,30,20,...,36,47,38,47,44,83,95,48,47,34
2,110001505000004,222,108,114,5,4,9,14,7,5,...,11,15,26,11,11,33,37,25,23,17
3,110001505000006,785,408,377,36,33,34,41,42,27,...,63,61,75,71,53,124,92,88,57,40
4,110001505000007,748,373,375,28,25,33,27,24,31,...,52,54,52,51,61,110,116,93,63,46


O que observamos da base setores:

| Coluna              | Interpretação                                                      |
| ------------------- | ------------------------------------------------------------------ |
| `CD_setor`          | Código único do setor censitário (15 dígitos)                      |
| `V01006` a `V01041` | Variáveis demográficas (idade, renda, etc. – checar no dicionário) |


Sabemos que:

Os 7 primeiros dígitos do CD_setor representam o município → compatível com ivs_filtrado["municipio"]

Os 9 primeiros dígitos representam o distrito → compatível com CD_DIST da base de distritos

Próximo passo: Integrar a base fazendo

1-Extrair os 7 primeiros dígitos de CD_setor para formar o campo CD_MUN

2-Unir com a base principal ivs_filtrado pela coluna municipio

3-Assim, teremos a base IVS/IDHM com dados de setores censitários incluídos

In [ ]:
import pandas as pd


In [ ]:
setores = pd.read_excel("Agregados_por_setores_demografia_BR.xlsx")
setores.head()


,CD_setor,V01006,V01007,V01008,V01009,V01010,V01011,V01012,V01013,V01014,...,V01032,V01033,V01034,V01035,V01036,V01037,V01038,V01039,V01040,V01041
0,110001505000002,928,428,500,30,39,24,44,36,25,...,68,62,88,68,58,144,129,124,66,53
1,110001505000003,556,270,286,15,20,20,19,30,20,...,36,47,38,47,44,83,95,48,47,34
2,110001505000004,222,108,114,5,4,9,14,7,5,...,11,15,26,11,11,33,37,25,23,17
3,110001505000006,785,408,377,36,33,34,41,42,27,...,63,61,75,71,53,124,92,88,57,40
4,110001505000007,748,373,375,28,25,33,27,24,31,...,52,54,52,51,61,110,116,93,63,46


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving atlasivs_dadosbrutos_pt_v2.xlsx to atlasivs_dadosbrutos_pt_v2 (1).xlsx


In [ ]:
# Reimportar pandas se necessário
import pandas as pd

# Recarregar a base IVS
ivs = pd.read_excel("atlasivs_dadosbrutos_pt_v2 (1).xlsx")

# Filtrar apenas os anos desejados
ivs_filtrado = ivs[ivs["ano"].isin([2000, 2010, 2020])].copy()


In [ ]:
import pandas as pd


In [ ]:
setores = pd.read_excel("Agregados_por_setores_demografia_BR.xlsx")
setores.head()


,CD_setor,V01006,V01007,V01008,V01009,V01010,V01011,V01012,V01013,V01014,...,V01032,V01033,V01034,V01035,V01036,V01037,V01038,V01039,V01040,V01041
0,110001505000002,928,428,500,30,39,24,44,36,25,...,68,62,88,68,58,144,129,124,66,53
1,110001505000003,556,270,286,15,20,20,19,30,20,...,36,47,38,47,44,83,95,48,47,34
2,110001505000004,222,108,114,5,4,9,14,7,5,...,11,15,26,11,11,33,37,25,23,17
3,110001505000006,785,408,377,36,33,34,41,42,27,...,63,61,75,71,53,124,92,88,57,40
4,110001505000007,748,373,375,28,25,33,27,24,31,...,52,54,52,51,61,110,116,93,63,46


In [ ]:
# Reimportar a base IVS (se já tiver feito o upload do arquivo Atlasivs_DadosBrutos.xlsx)
ivs = pd.read_excel("atlasivs_dadosbrutos_pt_v2 (1).xlsx")

# Filtrar apenas os anos de interesse: 2000, 2010 e 2020
ivs_filtrado = ivs[ivs["ano"].isin([2000, 2010, 2020])].copy()


UTILIZEI TODA A RAM DISPONIVEL, ISSO FAZ COM QUE AO TENTAR RODAR O CODIGO PARA UNIR TODAS AS BASES, A SESSÃO FALHE E REINICIE, ASSIM TENDO QUE REINICIAR OS DADOS PARA IMPORTAÇÃO DE VARIAVEL E BASE DE DADOS

In [ ]:
setores["CD_setor"] = setores["CD_setor"].astype(str)
setores["CD_MUN"] = setores["CD_setor"].str[:7]
ivs_filtrado["municipio"] = ivs_filtrado["municipio"].astype(str)

ivs_com_setores = pd.merge(ivs_filtrado, setores, left_on="municipio", right_on="CD_MUN", how="inner")
ivs_com_setores.shape
ivs_com_setores.head()


NameError: name 'setores' is not defined